In [23]:
import random
from datetime import datetime, date, timedelta
import pandas as pd
import itertools
import json
import numpy

In [24]:
def getRandomTime(min_x, max_x, start_time):
    start_time = datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S:%f")
    # min and max duration of the activity %H:%M:%S
    min_duration = datetime.combine(date.min, datetime.strptime(min_x, "%H:%M:%S:%f").time())
    max_duration = datetime.combine(date.min, datetime.strptime(max_x, "%H:%M:%S:%f").time()) 
    # get a ramdom millisec value between min and max duration
    millisec = random.randint(0, int(((max_duration - min_duration).total_seconds())*1000))
    # sum the start date and the calculated random value
    min_x = datetime.strptime(min_x, '%H:%M:%S:%f')
    b = timedelta(hours= min_x.hour, minutes = min_x.minute, seconds = min_x.second, milliseconds = min_x.microsecond)
    random_duration = start_time + timedelta(milliseconds = millisec) + b
    return (random_duration.strftime('%Y-%m-%d %H:%M:%S:%f'), millisec+int(b.total_seconds()*1000))    

In [25]:
def calculateDuration(df, start_time):
    for index, row in df.iterrows():
        if index != 0:
            previous = df.loc[index -1, "End_time"]
            a = getRandomTime(row["Min"],row["Max"], previous)
            df.at[index, "Start_time"] = previous
            df.at[index, "End_time"] = a[0]
            df.at[index, "Duration"] = a[1]
        else:
            a = getRandomTime(row["Min"],row["Max"], start_time)
            df.at[index, "Start_time"] = start_time
            df.at[index, "End_time"] = a[0]
            df.at[index, "Duration"] = a[1]
    return(df)

In [26]:
def writeToFile(df, agent, name):
    lines = []
    data = {}
    agenttxt = "agent(" + agent + "){"
    lines.append(agenttxt)
    #lines.append(start_time)
    for index, row in df.iterrows():
        txt = "goto(" + row["Activity"] + ");"
        seconds = timedelta(milliseconds=row["Duration"])
        #to have milliseconds remove /100
        txt2 = "wait(" + str(round((seconds.seconds))) + ");"
        lines.append(txt)
        lines.append(txt2)
    lines.append("}")
    str1 = "".join(lines)
    data["input"] = str1
    with open(name, 'w') as f:
        json.dump(data, f)
    #with open(name, 'w') as f:
    #    f.write('\n'.join(lines))
    #f.close()

In [27]:
def goto_A():
    entities_A = ["A1", "A2", "A3", "R1"]
    random.shuffle(entities_A)
    for el in entities_A:
        day.append([[el,'00:01:00:00','00:03:00:00']])
def goto_B():
    entities_B = ["B1", "B2", "B3", "R2"]
    random.shuffle(entities_B)
    for el in entities_B:
        day.append([[el,'00:01:00:00','00:03:00:00']])
def goto_C():
    entities_C = ["C1", "C2", "C3", "C4", "R3"]
    random.shuffle(entities_C)
    for el in entities_C:
        day.append([[el,'00:01:00:00','00:03:00:00']])
def goto_D():
    entities_D = ["D1", "D2", "D3"]
    random.shuffle(entities_D)
    for el in entities_D:
        day.append([[el,'00:01:00:00','00:03:00:00']])

In [28]:
#compose a normal day
def build_normal_day(): 
    goto_A()
    goto_B()
    goto_C()
    goto_D()
    final = list(itertools.chain.from_iterable(day))
    dd = pd.DataFrame(final, columns=['Activity', 'Min', 'Max'])
    header_list=["Activity","Min", "Max", "Start_time", "End_time", "Duration"]
    dd = dd.reindex(columns = header_list) 
    dd = dd.fillna("")
    return(dd)

In [29]:
#compose a day
def build_shuffle_day(): 
    sh = [goto_A, goto_B, goto_C, goto_D]
    random.shuffle(sh)
    sh = random.sample(sh, k=random.randint(1, 4))
    for func in sh:
        func()
    final = list(itertools.chain.from_iterable(day))
    dd = pd.DataFrame(final, columns=['Activity', 'Min', 'Max'])
    header_list=["Activity","Min", "Max", "Start_time", "End_time", "Duration"]
    dd = dd.reindex(columns = header_list) 
    dd = dd.fillna("")
    return(dd)

In [30]:
agent = "Agent1"
start_time = datetime(2022, 1, 1, 8, 20, 0, 0).strftime('%Y-%m-%d %H:%M:%S:%f')

for var in range(10):
    day = []
    # print("Day", var)
    activityList = build_normal_day()
    # print(activityList)
    # print(".........")
    final = calculateDuration(activityList, start_time)
    name = "normal/input_normal_" + str(var) +".json"
    writeToFile(final, agent, name)

    day = []
    activityList = build_shuffle_day()
    final2 = calculateDuration(activityList, start_time)
    name = "shuffle/input_shuffle_" + str(var) +".json"
    writeToFile(final2, agent, name)